In [1]:
# Evaluation notebook for kNN-based job recommendations using session embeddings

import os
import pickle
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import matplotlib.pyplot as plt

from collections import Counter

C:\Users\romai\anaconda3\Lib\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


In [2]:
# -------------------------------
# Evaluation functions
# -------------------------------


In [3]:
def compute_accuracy_metrics(recs, truth, k=20):
    #print(recs)
    #print(type(recs))
    #for i in recs:
    #    print(type(i))
    #print(truth)
    hits = [1 if item in truth else 0 for item in recs[:k]]
    #print(hits)
    #print("len(hits)")
    #print(len(hits))
    #for idx, hit in enumerate(hits):
        #print(idx, hit)
    dcg = sum([hit / np.log2(1+(idx+1)) for idx, hit in enumerate(hits)])
    idcg = sum([1.0 / np.log2(1+(i+1)) for i in range(min(len(truth), k))])
    ndcg = dcg / idcg if idcg > 0 else 0 # Normalized Discounted Cumulative Gain (nDCG)

    mrr = 0 # Mean Reciprocal Rank (MRR) (also referred to as Average Reciprocal Hit Rate (ARHR))
    for idx, item in enumerate(recs[:k]):
        if item in truth:
            mrr = 1.0 / (idx + 1)
            break

    return ndcg, mrr


def compute_fullsession_mrr(preds, truth, k=20):
    print(truth)
    print(preds)
    reciprocal_ranks = []
    for item in truth:
        #try:
        rank = preds[:k].index(item) + 1
        print(rank)
        reciprocal_ranks.append(1.0 / rank)
        #except ValueError:
            #print("error full session MRR")
            #continue
    return np.mean(reciprocal_ranks) if reciprocal_ranks else 0

In [4]:
def compute_fullsession_mrr(preds, truth, k=20):
    reciprocal_ranks = [1.0 / (preds[:k].index(item) + 1)
                         for item in truth if item in preds[:k]]
    return np.mean(reciprocal_ranks) if reciprocal_ranks else 0

In [5]:
def compute_epc(recs, truth , popularity, k=20):
    """
    print("def compute_epc(recs, truth , popularity, k=20):")
    print(" *"*20)
    print(" *"*20)
    print("recs")
    print(recs)
    print(type(recs))
    for i in recs:
        print(type(i))
    print(" *"*20)
    print("truth")
    print(truth)
    print(type(truth))
    for i in truth:
        print(type(i))
    """
    pop_max = sum(popularity.values())
    #print("pop_max = sum(popularity.values())")
    #print(pop_max)
    novelty = 0.0

    for i, item in enumerate(recs[:k]):
        #print("for i, item in enumerate(recs[:k]):")
        #print(i, item)
        #print(type(i), type(item))
        disc = 1 / np.log2(i + 2)
        p_rel = 1.0 if item in truth  else 0.0
        p_seen = np.log2(popularity.get(str(item), 1) + 1) / np.log2(pop_max + 1)
        novelty += disc * p_rel * (1 - p_seen)

    return novelty / k                   # ≈ divide by |R_k|


In [6]:
def compute_epd(recs, truth, item_vectors, k=20):
    #print(item_vectors)
    novelty = 0.0
    
    #count=0
    for i, r_i in enumerate(recs[:k]):
        disc = 1 / np.log2(i + 2)
        p_rel = 1.0 if r_i in truth else 0.0
        """
        if p_rel>0:
            print(i, k, r_i, count)
            count+=1
        
        print("p_rel = 1.0 if r_i in truth else 0.0")
        print(r_i)
        print(type(r_i))
        print(truth)
        print(type(truth))
        for i in truth:
            print(type(i))
        """
        distances = []
        for t in truth:
            # Flatten any tuples in session history
            if isinstance(t, tuple):
                for sub_t in t:
                    if r_i in item_vectors and sub_t in item_vectors:
                        sim = cosine_similarity([item_vectors[r_i]], [item_vectors[sub_t]])[0][0]
                        dist = 1 - sim
                        distances.append(dist)
                    else:
                        print("error during EPD : loop 1")
            else:
                if r_i in item_vectors and t in item_vectors:
                    """
                    print("if r_i in item_vectors and t in item_vectors:")
                    print(t)
                    print(type(t))
                    """
                    sim = cosine_similarity([item_vectors[r_i]], [item_vectors[t]])[0][0]
                    """
                    print("sim = cosine_similarity([item_vectors[r_i]], [item_vectors[t]])")
                    print(cosine_similarity([item_vectors[r_i]], [item_vectors[t]]))
                    print("sim = cosine_similarity([item_vectors[r_i]], [item_vectors[t]])[0]")
                    print(cosine_similarity([item_vectors[r_i]], [item_vectors[t]])[0])
                    print("sim = cosine_similarity([item_vectors[r_i]], [item_vectors[t]])[0][0]")
                    print(cosine_similarity([item_vectors[r_i]], [item_vectors[t]])[0][0])
                    """
                    dist = 1 - sim
                    #print(dist)
                    distances.append(dist)
                else:
                    print("r_i")
                    print(r_i)
                    print(type(r_i))
                    print("t")
                    print(t)
                    print(type(t))
                    print("error during EPD : loop 2")

        if distances:
            avg_dist = np.mean(distances)        # ≈ divide by |H_s|
            novelty += disc * p_rel * avg_dist
        else:
            print("error during EPD : if 3")

    return novelty / k                   # ≈ divide by |R_k|

In [7]:
def evaluate_all_metrics(preds, truth, history, item_vectors, popularity, k=20):
    #print(truth)
    # Flatten truth if it contains tuples
    flat_truth = []
    for t in truth:
        if isinstance(t, tuple):
            flat_truth.extend(t)
        else:
            flat_truth.append(t)
    #print(truth)
    
    # Flatten history if it contains tuples
    flat_history = []
    for t in history:
        if isinstance(t, tuple):
            flat_history.extend(t)
        else:
            flat_history.append(t)
    
    ## REMAINING JOBS PREDICTION
    ndcg, _ = compute_accuracy_metrics(preds, flat_truth, k)
    mrr = compute_fullsession_mrr(preds, flat_truth, k)
    epc = compute_epc(recs=preds, truth=flat_truth, popularity=popularity, k=k)
    epd = compute_epd(preds, flat_truth, item_vectors, k)
    
    ## NEXT ITEM PREDICTION
    # Use only the first relevant item as the next-item target
    if isinstance(truth, list) and len(truth) > 0:
        truth = [truth[0]]
    
    ndcg_next, mrr_next = compute_accuracy_metrics(preds, truth, k)
    epc_next = compute_epc(recs=preds, truth=truth, popularity=popularity, k=k)
    epd_next = compute_epd(recs, truth, item_vectors, k)
    
    return {
        'nDCG_next': ndcg_next,
        'MRR_next': mrr_next,
        'EPC_next': epc_next,
        'EPD_next': epd_next,
        f'nDCG_remaining': ndcg,
        f'MRR_remaining': mrr,
        f'EPC_remaining': epc,
        f'EPD_remaining': epd
    }


In [8]:
def extract_job_ids(series):
    all_items = []
    for x in series:
        try:
            items = eval(x) if isinstance(x, str) else x
            if not isinstance(items, list):
                items = [items]
            for entry in items:
                tokens = str(entry).split(',')
                tokens = [token.strip().strip('()') for token in tokens]
                all_items.extend(tokens)
        except:
            continue
    return all_items


In [9]:
# -------------------------------
# Loading data
# -------------------------------

In [10]:
path =  "../../data/"
dataset = "cb12/"

interim_path = path + dataset + "interim/"

encoded_path = interim_path + "infer/"

prediction_path = path + dataset + "predictions/"

evaluation_path = path + dataset + "evaluation/"

os.makedirs(os.path.dirname(interim_path), exist_ok=True)
os.makedirs(os.path.dirname(prediction_path), exist_ok=True)
os.makedirs(os.path.dirname(evaluation_path), exist_ok=True)

In [11]:
# load the item vectors
with open(os.path.join(interim_path, 'item_encodings.pkl'), 'rb') as f:
    item_vectors = pickle.load(f)


In [12]:
os.path.join(interim_path, 'item_encodings.pkl')

'../../data/cb12/interim/item_encodings.pkl'

In [13]:
item_vectors

{22.0: array([ 0.03123405, -0.05880071, -0.02361574, ...,  0.        ,
         0.        ,  0.        ]),
 60.0: array([-0.02298636,  0.05447083, -0.01613714, ...,  0.        ,
         0.        ,  0.        ]),
 81.0: array([-0.03808655,  0.08015087, -0.05203887, ...,  0.        ,
         0.        ,  0.        ]),
 244.0: array([-0.01253494,  0.01051599, -0.00906933, ...,  0.        ,
         0.        ,  0.        ]),
 370.0: array([-0.0232529 ,  0.03717544, -0.02286714, ...,  0.        ,
         0.        ,  0.        ]),
 408.0: array([-0.03603259,  0.02686551,  0.00423068, ...,  0.        ,
         0.        ,  0.        ]),
 413.0: array([-0.11095387,  0.03613185,  0.08677447, ...,  0.        ,
         0.        ,  0.        ]),
 414.0: array([-0.09694532,  0.04675978,  0.05777395, ...,  0.        ,
         0.        ,  0.        ]),
 441.0: array([ 0.02420265, -0.00323223,  0.00871207, ...,  0.        ,
         0.        ,  0.        ]),
 444.0: array([-0.05657297,  0.

In [14]:
len(item_vectors.keys())

34880

In [15]:
item_vectors[int(809208)]

array([-0.02313369, -0.08160122,  0.02220637, ...,  0.        ,
        0.        ,  0.        ])

In [16]:
for i in item_vectors.keys():
    print(i)
    print(type(i))

22.0
<class 'numpy.float64'>
60.0
<class 'numpy.float64'>
81.0
<class 'numpy.float64'>
244.0
<class 'numpy.float64'>
370.0
<class 'numpy.float64'>
408.0
<class 'numpy.float64'>
413.0
<class 'numpy.float64'>
414.0
<class 'numpy.float64'>
441.0
<class 'numpy.float64'>
444.0
<class 'numpy.float64'>
445.0
<class 'numpy.float64'>
458.0
<class 'numpy.float64'>
460.0
<class 'numpy.float64'>
465.0
<class 'numpy.float64'>
470.0
<class 'numpy.float64'>
481.0
<class 'numpy.float64'>
482.0
<class 'numpy.float64'>
510.0
<class 'numpy.float64'>
511.0
<class 'numpy.float64'>
519.0
<class 'numpy.float64'>
532.0
<class 'numpy.float64'>
544.0
<class 'numpy.float64'>
556.0
<class 'numpy.float64'>
745.0
<class 'numpy.float64'>
755.0
<class 'numpy.float64'>
761.0
<class 'numpy.float64'>
769.0
<class 'numpy.float64'>
778.0
<class 'numpy.float64'>
957.0
<class 'numpy.float64'>
1108.0
<class 'numpy.float64'>
1116.0
<class 'numpy.float64'>
1121.0
<class 'numpy.float64'>
1285.0
<class 'numpy.float64'>
1348.0
<c

In [17]:
# -------------------------------
# Load item popularity from training set
# -------------------------------
training_encoded = pd.read_csv(os.path.join(encoded_path, 'content_ae_encoder_train.csv'), sep='\t')
item_popularity = dict(Counter(extract_job_ids(training_encoded['items'])))

In [18]:
item_popularity

{'809208': 3,
 '136489': 1,
 '617374': 1,
 '206046': 2,
 '787741': 3,
 '663552': 2,
 '1083186': 1,
 '516837': 2,
 '507614': 1,
 '754917': 1,
 '686406': 2,
 '1058896': 1,
 '335132': 3,
 '576958': 3,
 '848187': 5,
 '602298': 2,
 '2121': 3,
 '586119': 1,
 '785425': 1,
 '97989': 1,
 '425464': 1,
 '883817': 1,
 '816846': 3,
 '13175': 1,
 '758079': 1,
 '148976': 1,
 '1032422': 1,
 '634527': 1,
 '1088828': 2,
 '455213': 1,
 '775441': 2,
 '155390': 1,
 '1021186': 1,
 '1021171': 3,
 '167313': 1,
 '500312': 3,
 '366888': 1,
 '449169': 1,
 '153632': 1,
 '1008042': 1,
 '680718': 1,
 '473911': 1,
 '403896': 1,
 '600033': 3,
 '802921': 3,
 '300053': 6,
 '497069': 1,
 '600058': 2,
 '283949': 3,
 '1066757': 1,
 '627377': 4,
 '812337': 3,
 '1078274': 3,
 '196603': 4,
 '146817': 3,
 '946506': 1,
 '271546': 1,
 '850543': 1,
 '168265': 1,
 '585922': 1,
 '892070': 1,
 '13914': 1,
 '1008855': 1,
 '613741': 1,
 '1035414': 2,
 '1046605': 1,
 '562908': 1,
 '900082': 2,
 '222539': 1,
 '187357': 1,
 '234872': 1,

In [19]:
try:
        item_popularity[809208]
except Exception as e:
                    print(f"Error in for loop: {e}")
                   

Error in for loop: 809208


In [20]:
item_popularity["809208"], type(item_popularity["809208"])

(3, int)

In [21]:
for i in item_popularity.values():
    print(type(i))

<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class 'int'>
<class

In [22]:
# -------------------------------
# Evaluation Loop
# -------------------------------

In [23]:
filename = os.path.join(prediction_path, f"ae_top_20.csv")
prediction_file = pd.read_csv(filename, sep='\t')
prediction_file.head()

,session_id,latent_session_vector,input_items,remaining_items,top_20_recommended_jobs_k=10,top_20_recommended_jobs_k=20,top_20_recommended_jobs_k=30,top_20_recommended_jobs_k=40,top_20_recommended_jobs_k=50,top_20_recommended_jobs_k=60,top_20_recommended_jobs_k=70,top_20_recommended_jobs_k=80,top_20_recommended_jobs_k=90,top_20_recommended_jobs_k=100
0,592,"0.0009184665,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0...",466171,"811982,1013337,565013,854862,277574","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436..."
1,592,"0.0017933721,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0...","466171,811982","1013337,565013,854862,277574","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,..."
2,592,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337","565013,854862,277574","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592..."
3,592,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013","854862,277574","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592..."
4,592,"0.001597002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013,854862",277574,"[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185..."


In [24]:
filename = os.path.join(prediction_path, f"dae_top_20.csv")
prediction_file = pd.read_csv(filename, sep='\t')
prediction_file.head()

,session_id,latent_session_vector,input_items,remaining_items,top_20_recommended_jobs_k=10,top_20_recommended_jobs_k=20,top_20_recommended_jobs_k=30,top_20_recommended_jobs_k=40,top_20_recommended_jobs_k=50,top_20_recommended_jobs_k=60,top_20_recommended_jobs_k=70,top_20_recommended_jobs_k=80,top_20_recommended_jobs_k=90,top_20_recommended_jobs_k=100
0,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....",466171,"811982,1013337,565013,854862,277574","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436..."
1,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982","1013337,565013,854862,277574","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,...","[38565, 75685, 327436, 837744, 854862, 473827,..."
2,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337","565013,854862,277574","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592..."
3,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013","854862,277574","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592...","[38565, 327436, 854862, 837744, 1106194, 35592..."
4,592,"0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0....","466171,811982,1013337,565013,854862",277574,"[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185...","[38565, 327436, 837744, 1106194, 355928, 82185..."


In [25]:
filename = os.path.join(prediction_path, f"vae_top_20.csv")
prediction_file = pd.read_csv(filename, sep='\t')
prediction_file.head()

,session_id,latent_session_vector,input_items,remaining_items,top_20_recommended_jobs_k=10,top_20_recommended_jobs_k=20,top_20_recommended_jobs_k=30,top_20_recommended_jobs_k=40,top_20_recommended_jobs_k=50,top_20_recommended_jobs_k=60,top_20_recommended_jobs_k=70,top_20_recommended_jobs_k=80,top_20_recommended_jobs_k=90,top_20_recommended_jobs_k=100
0,592,"0.0017225035,0.053339392,0.1214104,0.020633923...",466171,"811982,1013337,565013,854862,277574","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436...","[473827, 38565, 75685, 1046408, 252619, 327436..."
1,592,"0.07972506,-0.0036245026,0.12704223,-0.0136387...","466171,811982","1013337,565013,854862,277574","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,...","[38565, 854862, 75685, 327436, 837744, 473827,..."
2,592,"0.09760852,-0.05731277,0.17136127,-0.021898398...","466171,811982,1013337","565013,854862,277574","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592..."
3,592,"0.123271,-0.10945652,0.18692605,-0.026360432,-...","466171,811982,1013337,565013","854862,277574","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592...","[38565, 854862, 327436, 837744, 1106194, 35592..."
4,592,"0.13334964,0.0020997077,0.18070473,-0.04945600...","466171,811982,1013337,565013,854862",277574,"[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928...","[38565, 327436, 837744, 75685, 1106194, 355928..."


In [27]:
import ast

final_results = []

for ae in ['ae', 'dae', 'vae']:
    
    filename = os.path.join(prediction_path, f"{ae}_top_20.csv")
    print(filename)
    print(" *" * 20)
    try:
        prediction_file = pd.read_csv(filename, sep='\t')
        #display(prediction_file.head())
    except Exception as e:
        print(f"Failed to load {filename}: {e}")
        continue
            
    for k in range(10, 110, 10): #110, 10):
        print(" *" * 20)
        print("k =", k)
        print(" *" * 20)
    
        
        prediction_file[f"top_20_recommended_jobs_k={k}"] = prediction_file[f"top_20_recommended_jobs_k={k}"].apply(ast.literal_eval)

        for t in range(20, 0, -1): #21, 1):
            print(" =" * 20)
            print("k =", k)
            print("top_n =", t)
            print(" =" * 20)
            
            prediction_file[f"top_{t}_recommended_jobs_k={k}"] = prediction_file[f"top_20_recommended_jobs_k={k}"].apply(lambda x: x[:t])
            
            evaluation_results = []
            for index, row in prediction_file.iterrows():
                try:
                    #recs = eval(row[f"top_{t}_recommended_jobs_k={k}"])
                    recs = row[f"top_{t}_recommended_jobs_k={k}"]
                    truth = eval(row['remaining_items'])
                    history = eval(row['input_items'])

                    if not isinstance(recs, list): recs = [recs]
                    if not isinstance(truth, list): truth = [truth]
                    if not isinstance(history, list): history = [history]
                    
                    #recs = recs[:t]
                    #print(recs)
                    
                except Exception as e:
                    print(f"Error in for loop: {e}")
                    print("error in for loop")
                    continue

                metrics = evaluate_all_metrics(recs, truth, history, item_vectors, item_popularity, k)
                evaluation_results.append(metrics)

            if evaluation_results:
                df = pd.DataFrame(evaluation_results)
                all_recommended = set([item for rec in df.index for item in prediction_file.iloc[rec][f"top_{t}_recommended_jobs_k={k}"][:k]])
                #print(all_recommended)
                all_ground_truth = set()
                #print(prediction_file['remaining_items'])
                for sess in prediction_file['remaining_items']:
                    #print(sess)
                    tokens = str(sess).split(',')
                    tokens = [token.strip().strip('()') for token in tokens]
                    all_ground_truth.update(tokens)
                    #print(all_ground_truth)
                # (1) on the full dataset, i.e., how many of all available jobs can we recommend
                catalog_coverage = len(all_recommended) / len(item_vectors) 
                # (2) on the test dataset, i.e., how many of the jobs can we recommend that we expect to be interacted with during a session.
                all_ground_truth = set(map(int, all_ground_truth))
                testset_coverage = len(all_recommended & all_ground_truth) / len(all_ground_truth)
                
                #display(df.head())
                #display(df.describe())

                summary = df.mean(numeric_only=True).to_dict()
                #print(summary)
                summary.update({
                    'encoder_type': ae,
                    'top_n': t,
                    'k': k,
                    'CatalogCoverage (%)': round(catalog_coverage * 100, 2),
                    'TestsetCoverage (%)': round(testset_coverage * 100, 2)
                })
                #print(summary)
                final_results.append(summary)
                
        
        prediction_file_copy = prediction_file.copy()
        prediction_file = prediction_file_copy


../../data/cb12/predictions/ae_top_20.csv
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k = 10
 * * * * * * * * * * * * * * * * * * * *
 = = = = = = = = = = = = = = = = = = = =
k = 10
top_n = 20
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 10
top_n = 19
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 10
top_n = 18
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 10
top_n = 17
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 10
top_n = 16
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 10
top_n = 15
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 10
top_n = 14
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 10
top_n = 13
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = =

 = = = = = = = = = = = = = = = = = = = =
k = 40
top_n = 2
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 40
top_n = 1
 = = = = = = = = = = = = = = = = = = = =
 * * * * * * * * * * * * * * * * * * * *
k = 50
 * * * * * * * * * * * * * * * * * * * *
 = = = = = = = = = = = = = = = = = = = =
k = 50
top_n = 20
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 50
top_n = 19
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 50
top_n = 18
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 50
top_n = 17
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 50
top_n = 16
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 50
top_n = 15
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 50
top_n = 14
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = 

 = = = = = = = = = = = = = = = = = = = =
k = 80
top_n = 3
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 80
top_n = 2
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 80
top_n = 1
 = = = = = = = = = = = = = = = = = = = =
 * * * * * * * * * * * * * * * * * * * *
k = 90
 * * * * * * * * * * * * * * * * * * * *
 = = = = = = = = = = = = = = = = = = = =
k = 90
top_n = 20
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 90
top_n = 19
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 90
top_n = 18
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 90
top_n = 17
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 90
top_n = 16
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 90
top_n = 15
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = =

 = = = = = = = = = = = = = = = = = = = =
k = 20
top_n = 5
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 20
top_n = 4
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 20
top_n = 3
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 20
top_n = 2
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 20
top_n = 1
 = = = = = = = = = = = = = = = = = = = =
 * * * * * * * * * * * * * * * * * * * *
k = 30
 * * * * * * * * * * * * * * * * * * * *
 = = = = = = = = = = = = = = = = = = = =
k = 30
top_n = 20
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 30
top_n = 19
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 30
top_n = 18
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 30
top_n = 17
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = =

 = = = = = = = = = = = = = = = = = = = =
k = 60
top_n = 6
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 60
top_n = 5
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 60
top_n = 4
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 60
top_n = 3
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 60
top_n = 2
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 60
top_n = 1
 = = = = = = = = = = = = = = = = = = = =
 * * * * * * * * * * * * * * * * * * * *
k = 70
 * * * * * * * * * * * * * * * * * * * *
 = = = = = = = = = = = = = = = = = = = =
k = 70
top_n = 20
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 70
top_n = 19
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 70
top_n = 18
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = 

 = = = = = = = = = = = = = = = = = = = =
k = 100
top_n = 7
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 100
top_n = 6
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 100
top_n = 5
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 100
top_n = 4
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 100
top_n = 3
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 100
top_n = 2
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 100
top_n = 1
 = = = = = = = = = = = = = = = = = = = =
../../data/cb12/predictions/vae_top_20.csv
 * * * * * * * * * * * * * * * * * * * *
 * * * * * * * * * * * * * * * * * * * *
k = 10
 * * * * * * * * * * * * * * * * * * * *
 = = = = = = = = = = = = = = = = = = = =
k = 10
top_n = 20
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = 

 = = = = = = = = = = = = = = = = = = = =
k = 40
top_n = 9
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 40
top_n = 8
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 40
top_n = 7
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 40
top_n = 6
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 40
top_n = 5
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 40
top_n = 4
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 40
top_n = 3
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 40
top_n = 2
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 40
top_n = 1
 = = = = = = = = = = = = = = = = = = = =
 * * * * * * * * * * * * * * * * * * * *
k = 50
 * * * * * * * * * * * * * * * * * * * *
 = = = = = = = = = =

 = = = = = = = = = = = = = = = = = = = =
k = 80
top_n = 10
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 80
top_n = 9
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 80
top_n = 8
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 80
top_n = 7
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 80
top_n = 6
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 80
top_n = 5
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 80
top_n = 4
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 80
top_n = 3
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 80
top_n = 2
 = = = = = = = = = = = = = = = = = = = =
 = = = = = = = = = = = = = = = = = = = =
k = 80
top_n = 1
 = = = = = = = = = = = = = = = = = = = =
 * * * * 

In [28]:
len(all_ground_truth), all_ground_truth

(1225,
 {299008,
  32769,
  327681,
  765958,
  919560,
  981000,
  1038347,
  862222,
  481295,
  911376,
  694292,
  1105947,
  481310,
  518174,
  260130,
  872484,
  927781,
  886822,
  221224,
  262187,
  497707,
  561197,
  241710,
  882734,
  708658,
  477236,
  874549,
  284724,
  452663,
  800828,
  258110,
  577599,
  794690,
  561218,
  657477,
  274502,
  845895,
  264264,
  194637,
  792654,
  63567,
  368723,
  845908,
  165973,
  1069142,
  51287,
  667735,
  327770,
  110682,
  313438,
  481375,
  561249,
  667747,
  411754,
  882796,
  583789,
  98412,
  278642,
  1015924,
  452724,
  600182,
  512914,
  352376,
  948345,
  880762,
  311420,
  778364,
  323710,
  682109,
  499839,
  626817,
  542850,
  186495,
  284797,
  278661,
  110726,
  434312,
  92297,
  1032331,
  692365,
  577680,
  368785,
  14482,
  600211,
  243856,
  39057,
  1015954,
  966808,
  882841,
  516251,
  77979,
  374941,
  1022107,
  264350,
  929951,
  575643,
  688290,
  794787,
  995492,
  10

In [29]:
all_recommended & {631}

set()

In [30]:
all_recommended & all_ground_truth

{353,
 426,
 2306,
 16128,
 27244,
 27292,
 28426,
 28626,
 32769,
 35367,
 38565,
 42838,
 54180,
 63952,
 67239,
 72258,
 77979,
 80965,
 81202,
 85765,
 92390,
 101168,
 109568,
 135148,
 147349,
 147842,
 148525,
 149255,
 150002,
 156538,
 156800,
 157213,
 173728,
 184712,
 187393,
 191945,
 212561,
 228322,
 228642,
 235304,
 241510,
 246863,
 246864,
 246889,
 260130,
 261441,
 267384,
 275764,
 275905,
 276009,
 277216,
 277804,
 281056,
 282082,
 282146,
 282190,
 282441,
 283201,
 285406,
 285437,
 295335,
 295557,
 298659,
 312176,
 314498,
 326833,
 328329,
 330380,
 334562,
 334737,
 338172,
 342752,
 347225,
 354232,
 357304,
 361728,
 368785,
 370918,
 388895,
 410254,
 434528,
 441776,
 445212,
 447215,
 450024,
 452663,
 452724,
 456550,
 458151,
 476025,
 481261,
 481375,
 491276,
 491385,
 501515,
 507808,
 508867,
 513190,
 517780,
 520998,
 521338,
 523991,
 541953,
 558866,
 561218,
 561840,
 567644,
 568877,
 573326,
 574879,
 575010,
 577929,
 582110,
 583426,


In [31]:
len(all_recommended), all_recommended

(603,
 {318,
  353,
  426,
  685,
  1209,
  1382,
  2306,
  3176,
  11498,
  11913,
  16128,
  17095,
  24204,
  24321,
  27244,
  27292,
  28426,
  28452,
  28611,
  28626,
  32357,
  32480,
  32769,
  34851,
  35367,
  35397,
  35562,
  36878,
  37836,
  38565,
  39210,
  39464,
  41317,
  42838,
  50113,
  54180,
  59137,
  63352,
  63386,
  63952,
  67239,
  67754,
  72258,
  72892,
  73536,
  76585,
  76818,
  77979,
  77993,
  79371,
  80386,
  80965,
  80968,
  81008,
  81202,
  81205,
  82220,
  85765,
  90529,
  92390,
  96167,
  96410,
  101168,
  103662,
  106427,
  109568,
  110571,
  110689,
  111364,
  129888,
  132835,
  133153,
  133432,
  135148,
  139950,
  142692,
  142693,
  144417,
  146351,
  147349,
  147842,
  148525,
  149255,
  150002,
  151457,
  152096,
  152410,
  155233,
  156146,
  156538,
  156800,
  157213,
  157952,
  158829,
  158998,
  161547,
  162939,
  167980,
  173173,
  173470,
  173628,
  173728,
  173805,
  174087,
  174341,
  175274,
  175821

In [32]:
len(all_recommended), len(item_vectors), len(all_recommended & all_ground_truth), len(all_ground_truth)

(603, 34880, 217, 1225)

In [33]:
# -------------------------------
# Save Final Results
# -------------------------------
if final_results:
    final_df = pd.DataFrame(final_results)
    final_df.to_csv(evaluation_path+"summary_all_metrics.csv", index=False)
    print("\nFinal aggregated results saved to summary_all_metrics.csv")


Final aggregated results saved to summary_all_metrics.csv


In [34]:
final_df

,nDCG_next,MRR_next,EPC_next,EPD_next,nDCG_remaining,MRR_remaining,EPC_remaining,EPD_remaining,encoder_type,top_n,k,CatalogCoverage (%),TestsetCoverage (%)
0,0.026155,0.020224,0.002303,-8.695465e-21,0.185111,0.197461,0.045614,0.021597,ae,20,10,9.27,58.86
1,0.026155,0.020224,0.002303,-8.695465e-21,0.185111,0.197461,0.045614,0.021597,ae,19,10,9.27,58.86
2,0.026155,0.020224,0.002303,-8.695465e-21,0.185111,0.197461,0.045614,0.021597,ae,18,10,9.27,58.86
3,0.026155,0.020224,0.002303,-8.695465e-21,0.185111,0.197461,0.045614,0.021597,ae,17,10,9.27,58.86
4,0.026155,0.020224,0.002303,-8.695465e-21,0.185111,0.197461,0.045614,0.021597,ae,16,10,9.27,58.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...
595,0.025169,0.021233,0.000222,-3.392890e-21,0.133933,0.203848,0.003326,0.001510,vae,5,100,5.85,46.69
596,0.023109,0.020168,0.000204,-1.359787e-21,0.121135,0.204927,0.002985,0.001354,vae,4,100,5.02,41.71
597,0.020816,0.018836,0.000183,5.484878e-23,0.105585,0.203386,0.002556,0.001149,vae,3,100,4.10,35.92
598,0.016969,0.016272,0.000150,1.040254e-21,0.083807,0.188757,0.001992,0.000880,vae,2,100,2.99,27.27


In [35]:
final_df.describe()

,nDCG_next,MRR_next,EPC_next,EPD_next,nDCG_remaining,MRR_remaining,EPC_remaining,EPD_remaining,top_n,k,CatalogCoverage (%),TestsetCoverage (%)
count,600.000000,600.000000,600.000000,6.000000e+02,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000,600.000000
mean,0.027381,0.021324,0.000691,-6.706423e-21,0.169619,0.193606,0.012567,0.005878,10.500000,55.000000,8.397900,55.455467
std,0.005774,0.002899,0.000609,1.695164e-20,0.045129,0.014401,0.011125,0.005201,5.771093,28.746779,3.700452,15.840713
min,0.011243,0.011243,0.000103,-8.009953e-20,0.057132,0.140237,0.001302,0.000552,1.000000,10.000000,1.450000,15.840000
25%,0.024418,0.020168,0.000304,-9.555899e-21,0.145845,0.183040,0.005546,0.002608,5.750000,30.000000,5.827500,46.940000
50%,0.028729,0.022022,0.000443,-2.421486e-21,0.181358,0.194874,0.008200,0.003827,10.500000,55.000000,8.640000,59.350000
75%,0.031818,0.023588,0.000804,2.105743e-22,0.205734,0.203386,0.014900,0.006989,15.250000,80.000000,10.730000,68.160000
max,0.035814,0.024843,0.002591,5.771569e-20,0.229394,0.222941,0.045614,0.021597,20.000000,100.000000,15.550000,76.080000


In [ ]:
break

In [ ]:
"""
C:\Users\romai\AppData\Local\Temp\ipykernel_18288\3659507828.py:31: PerformanceWarning: DataFrame is highly fragmented. 
This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns 
at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
prediction_file[f"top_{t}_recommended_jobs_k={k}"] = prediction_file[f"top_20_recommended_jobs_k={k}"].apply(lambda x: x[:t])
"""

In [36]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
final_df = pd.read_csv(evaluation_path+"summary_all_metrics.csv")
final_df.head()

In [ ]:
final_df.describe()

# Plots

In [ ]:
## Fig. 5

In [ ]:
"""
The fgures show the infuence of the neighborhood size k for picking top-k similar sessions when
comparing the three autoencoder variations on both interaction data and combined data. We find that the
recommendation accuracy converges when k is picked to be around 60 or more
"""

In [ ]:
import matplotlib.pyplot as plt

# Filter DataFrame for rows where top_n == 20
filtered_df = final_df[final_df['top_n'] == 20]

In [ ]:
filtered_df.head()

In [ ]:
# Set up the plot
plt.figure(figsize=(8, 6))

# Plot one line per encoder_type
for encoder in filtered_df['encoder_type'].unique():
    subset = filtered_df[filtered_df['encoder_type'] == encoder]
    plt.plot(subset['k'], subset['nDCG_next'], marker='o', label=encoder+"_comb_nlp_next")  # 'D' is for diamond (losange)
    
    plt.plot(subset['k'], subset['nDCG_remaining'], marker='x', label=encoder+"_comb_nlp_remaining")  # 'D' is for diamond (losange)

# Customize plot
plt.title('nDCG vs. k (top_n jobs = 20)')
plt.xlabel('k')
plt.ylabel('nDCG@20')
plt.legend(title='Encoder Type')
plt.grid(True)
plt.tight_layout()

# Show plot
plt.show()


In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

# Original data
data = {
    'k': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100],
    'AE_Int': [0.1205, 0.1230, 0.1233, 0.1234, 0.1234, 0.1234, 0.1234, 0.1234, 0.1234, 0.1234],
    'AE_Comb': [0.1205, 0.1235, 0.1245, 0.1247, 0.1247, 0.1247, 0.1247, 0.1247, 0.1247, 0.1247],
    'DAE_Int': [0.1205, 0.1230, 0.1234, 0.1235, 0.1235, 0.1235, 0.1235, 0.1235, 0.1235, 0.1235],
    'DAE_Comb': [0.1205, 0.1230, 0.1233, 0.1234, 0.1234, 0.1234, 0.1234, 0.1234, 0.1234, 0.1234],
    'VAE_Int': [0.1110, 0.1098, 0.1086, 0.1080, 0.1075, 0.1073, 0.1073, 0.1073, 0.1073, 0.1073],
    'VAE_Comb': [0.1205, 0.1235, 0.1240, 0.1242, 0.1242, 0.1242, 0.1242, 0.1242, 0.1242, 0.1242],
}

df = pd.DataFrame(data)
long_df = df.melt(id_vars='k', var_name='encoder_type', value_name='nDCG')
long_df['top_n'] = 20

# Split sets
filtered_df_Lacic = long_df
filtered_df = filtered_df

# Custom styles for Lacic data
custom_styles_lacic = {
    'AE_Comb': {'color': 'plum', 'marker': '^', 'linestyle': '-'},
    'VAE_Comb': {'color': 'purple', 'marker': 'o', 'linestyle': '-'},
    'DAE_Int': {'color': 'green', 'marker': '>', 'linestyle': '--'},
    'AE_Int': {'color': 'darkblue', 'marker': 's', 'linestyle': '-'},
    'DAE_Comb': {'color': 'gold', 'marker': 'o', 'linestyle': '--'},
    'VAE_Int': {'color': 'darkred', 'marker': 'x', 'linestyle': '--'},
}

# Dash styles for comb_nlp (filtered_df)
custom_linestyles = [
    (0, (1, 10)),   # loosely dotted
    (0, (1, 5)),    # dotted
    (0, (5, 10)),
    (0, (1, 10)),
    (0, (1, 5)),
    (0, (5, 10))
]

# Plot
plt.figure(figsize=(10, 6))

# Plot Lacic with specific colors, markers, linestyles
for encoder in filtered_df_Lacic['encoder_type'].unique():
    subset = filtered_df_Lacic[filtered_df_Lacic['encoder_type'] == encoder]
    style = custom_styles_lacic.get(encoder, {'color': 'gray', 'marker': '.', 'linestyle': '-'})
    plt.plot(
        subset['k'], subset['nDCG'],
        label=encoder + "_Lacic",
        color=style['color'],
        marker=style['marker'],
        linestyle=style['linestyle']
    )

# Markers, colors, and linestyles
markers = ['D', 'o', 's', '^', 'v', '*']
colors = ['blue', 'green', 'red', 'orange', 'purple', 'brown']
custom_linestyles = [
    (0, (1, 10)),   # loosely dotted
    (0, (1, 5)),    # dotted
    (0, (5, 10)),
    (0, (1, 10)),
    (0, (1, 5)),
    (0, (5, 10))
]
    
# Plot NLP/Comb versions (custom dash styles)
for i, encoder in enumerate(filtered_df['encoder_type'].unique()):
    subset = filtered_df[filtered_df['encoder_type'] == encoder]
    plt.plot(
        subset['k'], subset['nDCG_remaining'],
        label=encoder + "_comb_nlp",
        marker=markers[i % len(markers)],
        color=colors[i % len(colors)],
        linestyle=custom_linestyles[i % len(custom_linestyles)]
    )

# Beautify
plt.title('nDCG vs. k (top_n jobs = 20)')
plt.xlabel('k')
plt.ylabel('nDCG@20')
plt.grid(True)
plt.legend(title='Encoder Type')
plt.tight_layout()

# Save the plot
plt.savefig("Fig_5.png", dpi=300, bbox_inches='tight')  # Save as PNG

plt.show()


In [ ]:
# REMAINING JOBS

In [ ]:
## Table 4

In [ ]:
"""
Table 4 Prediction results (k = 20) of remaining jobs that will be subject to interaction within a session.
(Color table online)
"""

In [ ]:
subset_df = final_df[(final_df["top_n"] == 20) & (final_df["k"] == 60)]
display(subset_df)

In [ ]:
# NEXT JOB PREDICTION PROBLEM

In [ ]:
## Fig. 8

In [ ]:
"""
Fig. 8 nDCG results for diferent recommendation list sizes (i.e., values of k) when predicting the next
job in the session. On all three datasets, both our proposed VAE approaches achieve competitive results
concerning accuracy (i.e., nDCG) metrics
"""

In [ ]:
filtered_k60 = final_df[final_df["k"] == 60]
print(filtered_k60['encoder_type'].unique())
filtered_k60.head()

In [ ]:
filtered_k60.columns

In [ ]:
# Extraire les lignes et colonnes d'intérêt
ae_vals = filtered_k60.loc[filtered_k60['encoder_type'] == 'ae', ["nDCG_next", "MRR_next", "EPC_next"]].reset_index(drop=True)
dae_vals = filtered_k60.loc[filtered_k60['encoder_type'] == 'dae', ["nDCG_next", "MRR_next", "EPC_next"]].reset_index(drop=True)

# Vérifier qu'ils ont bien la même forme
if ae_vals.shape == dae_vals.shape:
    comparison = ae_vals == dae_vals
    differences = ae_vals.compare(dae_vals)
    print("✅ Comparaison réussie")
else:
    print(f"❌ Les DataFrames ne sont pas alignés : {ae_vals.shape} vs {dae_vals.shape}")
    comparison = None
    differences = None


In [ ]:
ae_vals.head()

In [ ]:
dae_vals.head()

In [ ]:
print(differences)


In [ ]:
rows_with_diffs = comparison[~comparison.all(axis=1)]
print("Lignes avec des différences :")
print(rows_with_diffs)


In [ ]:
import matplotlib.pyplot as plt

# Filter data for k = 60
filtered_k60 = final_df[final_df["k"] == 60]

# Set up the plot
plt.figure(figsize=(8, 6))

# Markers, colors, and linestyles
markers = ['D', 'o', 's', '^', 'v', '*']
colors = ['blue', 'green', 'red', 'orange', 'purple', 'brown']
custom_linestyles = [
    (0, (1, 10)),   # loosely dotted
    (0, (1, 5)),    # dotted
    (0, (5, 10)),
    (0, (1, 10)),
    (0, (1, 5)),
    (0, (5, 10))
]

# Plot one line per encoder_type
for i, encoder in enumerate(filtered_k60["encoder_type"].unique()):
    subset = filtered_k60[filtered_k60["encoder_type"] == encoder]
    plt.plot(subset["top_n"], subset["nDCG_next"],
             label=encoder + "_comb_nlp",
             marker=markers[i % len(markers)],
             color=colors[i % len(colors)],
             linestyle=custom_linestyles[i % len(custom_linestyles)]
            )
    plt.plot(subset["top_n"], subset["nDCG_remaining"],
             label=encoder + "_comb_nlp",
             marker=markers[(i+3) % len(markers)],
             color=colors[(i+3) % len(colors)],
             linestyle=custom_linestyles[(i+3) % len(custom_linestyles)]
            )
    
# Customize plot
plt.title('nDCG vs. top_n (k = 60)')
plt.xlabel('top_n')
plt.ylabel('nDCG_next')
plt.xticks(range(1, 21, 1))  # Force x-axis ticks from 1 to 20
plt.legend(title='Encoder Type')
plt.grid(True)
plt.tight_layout()

# Show plot
plt.show()


In [ ]:
Fig_8_lacic = pd.read_csv(r'../../data/Lacic/Synthetic_Baseline_Methods__k_60_.csv')
print(Fig_8_lacic['encoder_type'].unique())
print("len(Fig_8_lacic)")
print(len(Fig_8_lacic))
Fig_8_lacic

In [ ]:
import matplotlib.pyplot as plt

# Filter data for k = 60
Fig_8_lacic = Fig_8_lacic #final_df[final_df["k"] == 60]

# Set up the plot
plt.figure(figsize=(10, 6))

# Define custom styles like in the image
custom_styles = {
    "VAE_Comb":     {'marker': 's', 'linestyle': '--', 'color': 'navy'},
    "VAE_Int":      {'marker': '^', 'linestyle': '-',  'color': 'darkblue'},
    "Bayes":        {'marker': '>', 'linestyle': '--', 'color': 'cornflowerblue'},
    "sKNN":         {'marker': 'P', 'linestyle': (0, (3, 2)), 'color': 'yellowgreen'},
    "iKNN":         {'marker': 'p', 'linestyle': ':',  'color': 'olive'},
    "S-sKNN":       {'marker': 'h', 'linestyle': '-',  'color': 'peru'},
    "V-sKNN":       {'marker': 'D', 'linestyle': '-',  'color': 'goldenrod'},
    "GRU4Rec":      {'marker': '8', 'linestyle': (0, (1, 1)), 'color': 'navajowhite'},
    "POP":          {'marker': 'X', 'linestyle': '-',  'color': 'firebrick'},
    "BPR-MF":       {'marker': 's', 'linestyle': '-',  'color': 'pink'},
    "pRNN":         {'marker': '^', 'linestyle': '--', 'color': 'indigo'}
}

# Plot each encoder_type with style
for encoder in Fig_8_lacic["encoder_type"].unique():
    subset = Fig_8_lacic[Fig_8_lacic["encoder_type"] == encoder]
    style = custom_styles.get(encoder, {'marker': 'o', 'linestyle': '-', 'color': 'black'})
    plt.plot(
        subset["top_n"], subset["nDCG"],
        label=encoder,
        marker=style['marker'],
        linestyle=style['linestyle'],
        color=style['color'],
        linewidth=2
    )

# Markers, colors, and linestyles
markers = ['D', 'o', 's', '^', 'v', '*']
colors = ['blue', 'green', 'red', 'orange', 'purple', 'brown']
custom_linestyles = [
    (0, (1, 10)),   # loosely dotted
    (0, (1, 5)),    # dotted
    (0, (5, 10)),
    (0, (1, 10)),
    (0, (1, 5)),
    (0, (5, 10))
]

# Plot one line per encoder_type
for i, encoder in enumerate(filtered_k60["encoder_type"].unique()):
    subset = filtered_k60[filtered_k60["encoder_type"] == encoder]
    plt.plot(subset["top_n"], subset["nDCG_next"],
             label=encoder + "_comb_nlp",
             marker=markers[i % len(markers)],
             color=colors[i % len(colors)],
             linestyle=custom_linestyles[i % len(custom_linestyles)]
            )
    
# Axis & legend styling
plt.title('nDCG vs. top_n jobs (k sessions = 60)', fontsize=14)
plt.xlabel('top_n jobs', fontsize=12)
plt.ylabel('nDCG_next', fontsize=12)
plt.xticks(range(1, 21, 1))  # Show ticks from 1 to 20
plt.yticks(fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(title='Encoder Type', fontsize=9, title_fontsize=10, ncol=3, loc='upper left')
plt.tight_layout()

# Save the plot
plt.savefig("Fig_8.png", dpi=300, bbox_inches='tight')  # Save as PNG

plt.show()


In [ ]:
## Fig. 9

In [ ]:
"""
EPC results for diferent recommendation list sizes (i.e., values of k) when predicting the next job
in the session. On all three datasets, both our proposed VAE approaches achieve good results concerning
beyond-accuracy (i.e., EPC) metrics
"""

In [ ]:
import matplotlib.pyplot as plt

# Filter data for k = 60
filtered_k60 = final_df[final_df["k"] == 60]

# Set up the plot
plt.figure(figsize=(8, 6))

# Markers, colors, and linestyles
markers = ['D', 'o', 's', '^', 'v', '*']
colors = ['blue', 'green', 'red', 'orange', 'purple', 'brown']
custom_linestyles = [
    (0, (1, 10)),   # loosely dotted
    (0, (1, 5)),    # dotted
    (0, (5, 10)),
    (0, (1, 10)),
    (0, (1, 5)),
    (0, (5, 10))
]

# Plot one line per encoder_type
for i, encoder in enumerate(filtered_k60["encoder_type"].unique()):
    subset = filtered_k60[filtered_k60["encoder_type"] == encoder]
    plt.plot(subset["top_n"], subset["EPC_next"],
             label=encoder + "_comb_nlp",
             marker=markers[i % len(markers)],
             color=colors[i % len(colors)],
             linestyle=custom_linestyles[i % len(custom_linestyles)]
            )
    
# Customize plot
plt.title('EPC vs. top_n (k = 60)')
plt.xlabel('top_n')
plt.ylabel('EPC')
plt.xticks(range(1, 21, 1))  # Force x-axis ticks from 1 to 20
plt.legend(title='Encoder Type')
plt.grid(True)
plt.tight_layout()

# Show plot
plt.show()


In [ ]:
Fig_9_lacic = pd.read_csv(r'../../data/Lacic/Synthetic_EPC_Data__k_60_.csv')
print(Fig_9_lacic['encoder_type'].unique())
print("len(Fig_9_lacic)")
print(len(Fig_9_lacic))
Fig_9_lacic.head()

In [ ]:
import matplotlib.pyplot as plt

# Filter data for k = 60
Fig_9_lacic = Fig_9_lacic #final_df[final_df["k"] == 60]

# Set up the plot
plt.figure(figsize=(10, 6))

# Define custom styles like in the image
custom_styles = {
    "VAE_Comb":     {'marker': 's', 'linestyle': '--', 'color': 'navy'},
    "VAE_Int":      {'marker': '^', 'linestyle': '-',  'color': 'darkblue'},
    "Bayes":        {'marker': '>', 'linestyle': '--', 'color': 'cornflowerblue'},
    "sKNN":         {'marker': 'P', 'linestyle': (0, (3, 2)), 'color': 'yellowgreen'},
    "iKNN":         {'marker': 'p', 'linestyle': ':',  'color': 'olive'},
    "S-sKNN":       {'marker': 'h', 'linestyle': '-',  'color': 'peru'},
    "V-sKNN":       {'marker': 'D', 'linestyle': '-',  'color': 'goldenrod'},
    "GRU4Rec":      {'marker': '8', 'linestyle': (0, (1, 1)), 'color': 'navajowhite'},
    "POP":          {'marker': 'X', 'linestyle': '-',  'color': 'firebrick'},
    "BPR-MF":       {'marker': 's', 'linestyle': '-',  'color': 'pink'},
    "pRNN":         {'marker': '^', 'linestyle': '--', 'color': 'indigo'}
}

# Plot each encoder_type with style
for encoder in Fig_9_lacic["encoder_type"].unique():
    subset = Fig_9_lacic[Fig_9_lacic["encoder_type"] == encoder]
    style = custom_styles.get(encoder, {'marker': 'o', 'linestyle': '-', 'color': 'black'})
    plt.plot(
        subset["top_n"], subset["EPC"],
        label=encoder,
        marker=style['marker'],
        linestyle=style['linestyle'],
        color=style['color'],
        linewidth=2
    )

# Markers, colors, and linestyles
markers = ['D', 'o', 's', '^', 'v', '*']
colors = ['blue', 'green', 'red', 'orange', 'purple', 'brown']
custom_linestyles = [
    (0, (1, 10)),   # loosely dotted
    (0, (1, 5)),    # dotted
    (0, (5, 10)),
    (0, (1, 10)),
    (0, (1, 5)),
    (0, (5, 10))
]

# Plot one line per encoder_type
for i, encoder in enumerate(filtered_k60["encoder_type"].unique()):
    subset = filtered_k60[filtered_k60["encoder_type"] == encoder]
    plt.plot(subset["top_n"], subset["EPC_next"],
             label=encoder + "_comb_nlp",
             marker=markers[i % len(markers)],
             color=colors[i % len(colors)],
             linestyle=custom_linestyles[i % len(custom_linestyles)]
            )
    
# Axis & legend styling
plt.title('EPC vs. top_n jobs (k sessions = 60)', fontsize=14)
plt.xlabel('top_n jobs', fontsize=12)
plt.ylabel('EPC', fontsize=12)
plt.xticks(range(1, 21, 1))  # Show ticks from 1 to 20
plt.yticks(fontsize=10)
plt.grid(True, linestyle='--', alpha=0.6)
plt.legend(title='Encoder Type', fontsize=9, title_fontsize=10, ncol=3, loc='upper left')
plt.tight_layout()
# Save the plot
plt.savefig("Fig_9.png", dpi=300, bbox_inches='tight')  # Save as PNG

plt.show()


In [ ]:
final_df = pd.read_csv(evaluation_path+"summary_all_metrics.csv")
final_df.head()

In [ ]:
import matplotlib.pyplot as plt

# Filter DataFrame for rows where top_n == 20
filtered_df = final_df[final_df['top_n'] == 20].copy()

import matplotlib.pyplot as plt

# Filter data for k = 60
filtered_k60 = final_df[final_df["k"] == 60]

# Set up the plot
plt.figure(figsize=(8, 6))

# Markers, colors, and linestyles
markers = ['D', 'o', 's', '^', 'v', '*']
colors = ['blue', 'green', 'red', 'orange', 'purple', 'brown']
custom_linestyles = [
    (0, (1, 10)),   # loosely dotted
    (0, (1, 5)),    # dotted
    (0, (5, 10)),
    (0, (1, 10)),
    (0, (1, 5)),
    (0, (5, 10))
]

# Plot one line per encoder_type
for i, encoder in enumerate(filtered_k60["encoder_type"].unique()):
    subset = filtered_k60[filtered_k60["encoder_type"] == encoder]
    plt.plot(subset["top_n"], subset["nDCG_next"],
             label=encoder + "_comb_nlp_next",
             marker=markers[i % len(markers)],
             color=colors[i % len(colors)],
             linestyle=custom_linestyles[i % len(custom_linestyles)]
            )
    plt.plot(subset["top_n"], subset["nDCG_remaining"],
             label=encoder + "_comb_nlp_remaining",
             marker=markers[(i+3) % len(markers)],
             color=colors[(i+3) % len(colors)],
             linestyle=custom_linestyles[(i+3) % len(custom_linestyles)]
            )
    
# Customize plot
plt.title('nDCG vs. top_n (k = 60)')
plt.xlabel('top_n')
plt.ylabel('nDCG_next')
plt.xticks(range(1, 21, 1))  # Force x-axis ticks from 1 to 20
plt.legend(title='Encoder Type')
plt.grid(True)
plt.tight_layout()

# Show plot
plt.show()



In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook


# Set up the 3D plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Choose the metric: either nDCG_next or nDCG_remaining
metric = 'nDCG_next'  # or 'nDCG_remaining'

# Optional: filter to relevant encoders or loop over them
for i, encoder in enumerate(final_df["encoder_type"].unique()):
    subset = final_df[final_df["encoder_type"] == encoder]
    
    x = subset["top_n"]
    y = subset["k"]
    z = subset[metric]
    
    ax.scatter(x, y, z,
               label=encoder,
               marker='o',
               color=colors[i % len(colors)],
               s=40, alpha=0.7)

# Customize the plot
ax.set_title(f'3D View: {metric} vs. top_n & k')
ax.set_xlabel('top_n')
ax.set_ylabel('k')
ax.set_zlabel(metric)
ax.legend(title="Encoder Type")
plt.tight_layout()

# Show the 3D plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook


# Set up the 3D plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Choose the metric: either nDCG_next or nDCG_remaining
metric = 'nDCG_remaining'  # or 'nDCG_remaining'

# Optional: filter to relevant encoders or loop over them
for i, encoder in enumerate(final_df["encoder_type"].unique()):
    subset = final_df[final_df["encoder_type"] == encoder]
    
    x = subset["top_n"]
    y = subset["k"]
    z = subset[metric]
    
    ax.scatter(x, y, z,
               label=encoder,
               marker='o',
               color=colors[i % len(colors)],
               s=40, alpha=0.7)

# Customize the plot
ax.set_title(f'3D View: {metric} vs. top_n & k')
ax.set_xlabel('top_n')
ax.set_ylabel('k')
ax.set_zlabel(metric)
ax.legend(title="Encoder Type")
plt.tight_layout()

# Show the 3D plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook


# Set up the 3D plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Choose the metric: either nDCG_next or nDCG_remaining
metric = 'MRR_next'  # or 'nDCG_remaining'

# Optional: filter to relevant encoders or loop over them
for i, encoder in enumerate(final_df["encoder_type"].unique()):
    subset = final_df[final_df["encoder_type"] == encoder]
    
    x = subset["top_n"]
    y = subset["k"]
    z = subset[metric]
    
    ax.scatter(x, y, z,
               label=encoder,
               marker='o',
               color=colors[i % len(colors)],
               s=40, alpha=0.7)

# Customize the plot
ax.set_title(f'3D View: {metric} vs. top_n & k')
ax.set_xlabel('top_n')
ax.set_ylabel('k')
ax.set_zlabel(metric)
ax.legend(title="Encoder Type")
plt.tight_layout()

# Show the 3D plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D

%matplotlib notebook


# Set up the 3D plot
fig = plt.figure(figsize=(10, 7))
ax = fig.add_subplot(111, projection='3d')

# Choose the metric: either nDCG_next or nDCG_remaining
metric = 'MRR_next'  # or 'nDCG_remaining'

# Optional: filter to relevant encoders or loop over them
for i, encoder in enumerate(final_df["encoder_type"].unique()):
    subset = final_df[final_df["encoder_type"] == encoder]
    
    x = subset["top_n"]
    y = subset["k"]
    z = subset[metric]
    
    ax.scatter(x, y, z,
               label=encoder,
               marker='o',
               color=colors[i % len(colors)],
               s=40, alpha=0.7)

# Customize the plot
ax.set_title(f'3D View: {metric} vs. top_n & k')
ax.set_xlabel('top_n')
ax.set_ylabel('k')
ax.set_zlabel(metric)
ax.legend(title="Encoder Type")
plt.tight_layout()

# Show the 3D plot
plt.show()
